[Reference](https://medium.com/mlearning-ai/create-a-neural-network-with-pytorch-lightning-in-just-100-lines-of-code-43eccbf3fba)

In [2]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 10.5 MB/s 
     |████████████████████████████████| 125 kB 18.1 MB/s 
     |████████████████████████████████| 529 kB 63.0 MB/s 
     |████████████████████████████████| 87 kB 4.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=99bc29810149626961ab25572fe1ce4c9fdf068e52358481549bbfb5dc4ed96d
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [3]:
#%% packages
# data handling
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [4]:
#%% data import
data = fetch_california_housing()
X = data.data.astype(np.float32)
y = data.target
print(f"X shape: {X.shape}, y shape: {y.shape}")
scaler = StandardScaler()  # data scaling
X_scaled = scaler.fit_transform(X)
#%% splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=123, train_size=0.8)

X shape: (20640, 8), y shape: (20640,)


In [5]:
class LinearRegressionDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
class LitLinearRegression(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden1, learning_rate):
        super(LitLinearRegression, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden1)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden1, hidden1)
        self.relu = nn.ReLU()
        self.linear3 = nn.Linear(hidden1, output_dim)
        self.relu = nn.ReLU()
        self.loss_fun = nn.MSELoss()
        self.learning_rate = learning_rate
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y = y.type(torch.float32)
        # forward pass
        y_pred = self.forward(X).squeeze()
        # compute loss
        loss = self.loss_fun(y_pred, y)
        self.log_dict({'train_loss': loss}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y = y.type(torch.float32)
        # forward pass
        y_pred = self.forward(X).squeeze()        
        # compute metrics       
        print(y_pred) 
        r2 = r2_score(y_pred, y)
        loss = self.loss_fun(y_pred[0], y[0])
        self.log_dict({'test_loss': loss, 'r2': r2}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss


In [9]:
hidden1 = 20  #1 
max_epochs = 500  #2
lr = 0.1  #3
train_batch_size = 128  #4
test_batch_size =8192  #5

In [10]:
# model instance
input_dim = X_scaled.shape[1]
model = LitLinearRegression(input_dim=input_dim, output_dim=1, hidden1=hidden1, learning_rate=lr)

In [11]:
train_loader = DataLoader(dataset = LinearRegressionDataset(X_train, y_train), batch_size=train_batch_size)
test_loader = DataLoader(dataset = LinearRegressionDataset(X_test, y_test), batch_size=test_batch_size)

In [12]:
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=0.00, patience=20, verbose=True, mode="min")

In [13]:
trainer = pl.Trainer(accelerator='cpu', devices=1, max_epochs=max_epochs, callbacks=[early_stop_callback], log_every_n_steps=8)  #1
trainer.fit(model=model, train_dataloaders=train_loader)  #2

/usr/local/lib/python3.7/dist-packages/lightning_lite/accelerators/cuda.py:159: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type    | Params
-------------------------------------
0 | linear1  | Linear  | 180   
1 | relu     | ReLU    | 0     
2 | linear2  | Linear  | 420   
3 | linear3  | Linear  | 21    
4 | loss_fun | MSELoss | 0     
-------------------------------------
621       Trainable params
0         Non-trainable params
621       Total params
0.002     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved. New best score: 0.685
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.271 >= min_delta = 0.0. New best score: 0.414
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.045 >= min_delta = 0.0. New best score: 0.369
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.021 >= min_delta = 0.0. New best score: 0.348
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.007 >= min_delta = 0.0. New best score: 0.341
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.005 >= min_delta = 0.0. New best score: 0.336
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.007 >= min_delta = 0.0. New best score: 0.329
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.003 >= min_delta = 0.0. New best score: 0.326
INFO:pytorch_lightni

In [14]:
trainer.test(model=model, dataloaders=test_loader)

Testing: 0it [00:00, ?it/s]

tensor([1.7725, 1.4354, 1.6671,  ..., 0.8443, 1.8803, 3.5391])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           r2               0.6751212852982986
        test_loss           0.0657818540930748
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0657818540930748, 'r2': 0.6751212852982986}]